### 1 - Import test data

In [ ]:
import yfinance as yf
import pandas as pd

dataF = yf.download("EURUSD=X", start="2022-10-7", end="2022-12-5", interval='1h')
dataF.iloc[:,:]
#dataF.Open.iloc

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-10-07 00:00:00+01:00,0.979624,0.980104,0.978953,0.979144,0.979144,0
2022-10-07 01:00:00+01:00,0.979528,0.980008,0.978953,0.979624,0.979624,0
2022-10-07 02:00:00+01:00,0.979432,0.980104,0.979048,0.980008,0.980008,0
2022-10-07 03:00:00+01:00,0.979912,0.981451,0.979816,0.981065,0.981065,0
2022-10-07 04:00:00+01:00,0.981162,0.981354,0.980585,0.980777,0.980777,0
...,...,...,...,...,...,...
2022-12-02 18:00:00+00:00,1.052853,1.053519,1.052189,1.052189,1.052189,0
2022-12-02 19:00:00+00:00,1.052078,1.053852,1.051304,1.053741,1.053741,0
2022-12-02 20:00:00+00:00,1.053630,1.054185,1.053408,1.053630,1.053630,0


### 2 - Define your signal function

In [ ]:
def signal_generator(df):
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    previous_open = df.Open.iloc[-2]
    previous_close = df.Close.iloc[-2]

    # Bearish Pattern
    if (open>close and
    previous_open<previous_close and
    close<previous_open and
    open>=previous_close):
        return 1

    # Bullish Pattern
    elif (open<close and
        previous_open>previous_close and
        close>previous_open and
        open<=previous_close):
        return 2

    # No clear pattern
    else:
        return 0

signal = []
signal.append(0)
for i in range(1,len(dataF)):
    df = dataF[i-1:i+1]
    signal.append(signal_generator(df))
#signal_generator(data)
dataF["signal"] = signal

NameError: name 'data' is not defined

In [ ]:
dataF.signal.value_counts()
#dataF.iloc[:, :]

signal
0    1
Name: count, dtype: int64

### 3 - Connect to the market and execute trades

In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [ ]:
from alpaca.data.historical import CryptoHistoricalDataClient

# No keys required for crypto data
client = CryptoHistoricalDataClient()

In [ ]:
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame

# Creating request object
request_params = CryptoBarsRequest(
  symbol_or_symbols=["BTC/USD"],
  timeframe=TimeFrame.Day,
  start="2020-01-01",
  end="2024-04-07"
)

In [ ]:
# Retrieve daily bars for Bitcoin in a DataFrame and printing it
btc_bars = client.get_crypto_bars(request_params)

# Convert to dataframe
btc_bars.df

open        high         low  \
symbol  timestamp                                                      
BTC/USD 2021-01-01 06:00:00+00:00  29255.710  29682.2900  28707.5600   
        2021-01-02 06:00:00+00:00  29678.340  34200.0000  29555.9900   
        2021-01-03 06:00:00+00:00  33769.520  34812.9300  32300.6100   
        2021-01-04 06:00:00+00:00  32907.360  33496.0300  27900.0000   
        2021-01-05 06:00:00+00:00  30461.840  35851.4200  29927.5500   
...                                      ...         ...         ...   
        2024-04-02 05:00:00+00:00  66702.800  67171.2650  64458.5180   
        2024-04-03 05:00:00+00:00  66407.395  66910.0455  65114.5405   
        2024-04-04 05:00:00+00:00  65500.900  69342.9600  65398.2810   
        2024-04-05 05:00:00+00:00  67719.900  68622.7050  66024.9000   
        2024-04-06 05:00:00+00:00  67834.450  69737.7650  67613.6950   

                                        close       volume  trade_count  \
symbol  timestamp                                                         
BTC/USD 2021-01-01 06:00:00+00:00  29676.7900   848.874030      29639.0   
        2021-01-02 06:00:00+00:00  33769.5200  2144.592516      60152.0   
        2021-01-03 06:00:00+00:00  32908.0200  1838.695433      58725.0   
        2021-01-04 06:00:00+00:00  30441.5700  2711.189503      69226.0   
        2021-01-05 06:00:00+00:00  35063.0000  1756.751333      61880.0   
...                                       ...          ...          ...   
        2024-04-02 05:00:00+00:00  66377.6890    25.747100        266.0   
        2024-04-03 05:00:00+00:00  65539.6050    16.011660        138.0   
        2024-04-04 05:00:00+00:00  67640.8055     9.847211        338.0   
        2024-04-05 05:00:00+00:00  67731.5500     0.623981         88.0   
        2024-04-06 05:00:00+00:00  69357.5160     0.806721         64.0   

                                           vwap  
symbol  timestamp                                
BTC/USD 2021-01-01 06:00:00+00:00  29316.444625  
        2021-01-02 06:00:00+00:00  31941.412694  
        2021-01-03 06:00:00+00:00  33505.269474  
        2021-01-04 06:00:00+00:00  31267.965121  
        2021-01-05 06:00:00+00:00  33151.367357  
...                                         ...  
        2024-04-02 05:00:00+00:00  64707.073752  
        2024-04-03 05:00:00+00:00  65317.503263  
        2024-04-04 05:00:00+00:00  67790.435557  
        2024-04-05 05:00:00+00:00  67083.413190  
        2024-04-06 05:00:00+00:00  68256.051536  

[1192 rows x 7 columns]

In [ ]:
from config import access_token, accountID
def get_candles(n):
    #access_token='XXXXXXX'#you need token here generated from OANDA account
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M15)
    candles = collector.grab(n)
    return candles

candles = get_candles(3)
for candle in candles:
    print(float(str(candle.bid.o))>1)


ModuleNotFoundError: No module named 'config'

In [ ]:
def trading_job():
    candles = get_candles(3)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])

    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal = signal_generator(dfstream.iloc[:-1,:])#

    # EXECUTING ORDERS
    #accountID = "XXXXXXX" #your account ID here
    client = API(access_token)

    SLTPRatio = 2.
    previous_candleR = abs(dfstream['High'].iloc[-2]-dfstream['Low'].iloc[-2])

    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio

    print(dfstream.iloc[:-1,:])
    print(TPBuy, "  ", SLBuy, "  ", TPSell, "  ", SLSell)
    signal = 2
    #Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument="EUR_USD", units=-1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)
    #Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument="EUR_USD", units=1000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

### 4 - Executing orders automatically with a scheduler

In [ ]:
trading_job()

#scheduler = BlockingScheduler()
#scheduler.add_job(trading_job, 'cron', day_of_week='mon-fri', hour='00-23', minute='1,16,31,46', start_date='2022-01-12 12:00:00', timezone='America/Chicago')
#scheduler.start()

In [ ]:
from datetime import datetime
import pandas as pd
from alpaca_trade_api.rest import REST
from alpaca_trade_api.rest import TimeFrame

# Your Alpaca API credentials
API_KEY = 'CKJDS2I3H0AY0UUC2KXT'
API_SECRET = 'im4bogPdnmVenoh33KR4mpfnO9djjq64gfPCXAja'

api = REST(API_KEY, API_SECRET, base_url='https://broker-api.sandbox.alpaca.markets')



In [ ]:
dataF = yf.download("EURUSD=X", start="2022-10-7", end="2022-12-5", interval='1h')
dataF.iloc[:,:]


[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-10-07 00:00:00+01:00,0.979624,0.980104,0.978953,0.979144,0.979144,0
2022-10-07 01:00:00+01:00,0.979528,0.980008,0.978953,0.979624,0.979624,0
2022-10-07 02:00:00+01:00,0.979432,0.980104,0.979048,0.980008,0.980008,0
2022-10-07 03:00:00+01:00,0.979912,0.981451,0.979816,0.981065,0.981065,0
2022-10-07 04:00:00+01:00,0.981162,0.981354,0.980585,0.980777,0.980777,0
...,...,...,...,...,...,...
2022-12-02 18:00:00+00:00,1.052853,1.053519,1.052189,1.052189,1.052189,0
2022-12-02 19:00:00+00:00,1.052078,1.053852,1.051304,1.053741,1.053741,0
2022-12-02 20:00:00+00:00,1.053630,1.054185,1.053408,1.053630,1.053630,0


In [ ]:
def signal_generator(df):
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    previous_open = df.Open.iloc[-2]
    previous_close = df.Close.iloc[-2]

    # Bearish Pattern
    if (open>close and
    previous_open<previous_close and
    close<previous_open and
    open>=previous_close):
        return 1

       # Bullish Pattern
    elif (open < close and
        previous_open > previous_close and
        close > previous_open and
        open <= previous_close):
        return 2

    # No clear pattern
    else:
        return 0



In [ ]:
signal = []
signal.append(0)
for i in range(1, len(dataF)):
    df = dataF[i-1:i+1]
    signal.append(signal_generator(df))

dataF["signal"] = signal

dataF.signal.value_counts()



signal
0    863
2     59
1     52
Name: count, dtype: int64

In [ ]:
def get_candles(symbol, limit):
    candles = api.get_bars(symbol, '1h', limit=limit).df
    return candles


In [ ]:
def trading_job():
    candles = get_candles("EURUSD", 3)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])

    i=0
    for candle in candles:
        # dfstream.loc[i, ['Open']] = candle.o
        # dfstream.loc[i, ['Close']] = candle.c
        # dfstream.loc[i, ['High']] = candle.h
        # dfstream.loc[i, ['Low']] = candle.l
        dfstream.loc[i, ['Open']] = candle.open
        dfstream.loc[i, ['Close']] = candle.close
        dfstream.loc[i, ['High']] = candle.high
        dfstream.loc[i, ['Low']] = candle.low
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal = signal_generator(dfstream.iloc[:-1,:])

    account = api.get_account()
    SLTPRatio = 2.
    previous_candleR = abs(dfstream['High'].iloc[-2] - dfstream['Low'].iloc[-2])

    SLBuy = float(candle.o) - previous_candleR
    SLSell = float(candle.o) + previous_candleR

    TPBuy = float(candle.o) + previous_candleR * SLTPRatio
    TPSell = float(candle.o) - previous_candleR * SLTPRatio

    print(dfstream.iloc[:-1,:])
    print(TPBuy, "  ", SLBuy, "  ", TPSell, "  ", SLSell)
    signal = 2

    #Sell
    if signal == 1:
        api.submit_order(
            symbol="EURUSD",
            qty=1000,
            side="sell",
            type="market",
            time_in_force="gtc",
            order_class="bracket",
            take_profit=dict(limit_price=TPSell),
            stop_loss=dict(stop_price=SLSell)
        )

    #Buy
    elif signal == 2:
        api.submit_order(
            symbol="EURUSD",
            qty=1000,
            side="buy",
            type="market",
            time_in_force="gtc",
            order_class="bracket",
            take_profit=dict(limit_price=TPBuy),
            stop_loss=dict(stop_price=SLBuy)
        )

trading_job()


HTTPError: 403 Client Error: Forbidden for url: https://data.alpaca.markets/v2/stocks/EURUSD/bars?timeframe=1h&adjustment=raw&limit=3

In [ ]:
!curl -X GET \
    -H "APCA-API-KEY-ID: CKJDS2I3H0AY0UUC2KXT"  \
    -H "APCA-API-SECRET-KEY: im4bogPdnmVenoh33KR4mpfnO9djjq64gfPCXAja" \
    https://paper-api.alpaca.markets \

{"message": "endpoint not found."}
curl: (6) Could not resolve host: \


In [ ]:
import requests

url = "https://paper-api.alpaca.markets/v2/account"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{"message": "forbidden."}



In [ ]:
import alpaca_trade_api as tradeapi

# Assuming 'api' is already initialized with valid credentials
# api = tradeapi.REST()
api = tradeapi.REST(key_id='CKJDS2I3H0AY0UUC2KXT', secret_key='im4bogPdnmVenoh33KR4mpfnO9djjq64gfPCXAja')

# Check if EURUSD is a valid symbol
symbols = api.list_assets()
if 'EURUSD' not in symbols:
    print("EURUSD is not a valid symbol.")
else:
    print("EURUSD is a valid symbol.")

# Check if 1h is a valid timeframe
timeframes = api.list_timeframes()
if '1h' not in timeframes:
    print("1h is not a valid timeframe.")
else:
    print("1h is a valid timeframe.")

APIError: forbidden.

In [ ]:
api = tradeapi.REST(key_id='CKJDS2I3H0AY0UUC2KXT', secret_key='im4bogPdnmVenoh33KR4mpfnO9djjq64gfPCXAja')

# Get the account information
account = api.get_account()

# Print the account information
print(account)

APIError: forbidden.